In [1]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
import os
import pickle
import jieba
import json
import operator
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from datetime import datetime
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
font_path = 'msjh.ttc'
font = font_manager.FontProperties(fname='msjh.ttc',
                                   weight='bold',
                                   style='normal', size=16)

# import data_set

In [3]:
data = pd.read_excel('C:/Users/USER/github/bigdata_marketing/data_set/hw1_text.xlsx')
data.head(3)
userid = pd.read_excel('C:/Users/USER/github/bigdata_marketing/data_set/hw1_table.xlsx')
userid.head(3)

,90507,doc,Unnamed: 2,Unnamed: 3
0,1192367,gram,NaN,NaN
1,編號,詞,TF,DF
2,1,表示,100945,54184


# make an user_define name dict for jeiba package

In [4]:
useridlist = []
a = userid.iloc[2:,1]
b = pd.DataFrame(a)
for i in range(len(b)):
    useridlist.append(b.iloc[i,0])

# save the user_define dict into a txt file 

In [5]:
fileObject = open('sampleList.txt', 'w',encoding='utf-8')
for ip in useridlist:
    fileObject.write(ip)
    fileObject.write('\n')
fileObject.close()

# import jeiba package

In [6]:
jieba.set_dictionary('C:/Users/USER/github/bigdata_marketing/jieba_data/dict.txt.big')
jieba.load_userdict('C:/Users/USER/github/bigdata_marketing/tools/sampleList.txt')
stopwords = []
with open('C:/Users/USER/github/bigdata_marketing/jieba_data/stopwords.txt', 'r', encoding='UTF-8') as file:
    for each in file.readlines():
        stopwords.append(each.strip())
    stopwords.append(' ')

stopwords[:100]

Building prefix dict from C:\Users\USER\github\bigdata_marketing\jieba_data\dict.txt.big ...
Dumping model to file cache C:\Users\USER\AppData\Local\Temp\jieba.u05a435a89c46b8e0a56d077f738ce064.cache
Loading model cost 1.036 seconds.
Prefix dict has been built succesfully.


['\ufeff',
 '',
 '$',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96']

# clean the data set and cut the word

In [7]:
def remove_punctuation(content_string, user_define_list=False):
    if(user_define_list):
        punctuation = user_define_list
    else:
        punctuation=list("!@#$%^&*()_+=-[]`~'\"|/\\abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ,.;{}\r\xa0\u3000、，。「」！？；：<>12345678910")
        
    for word in punctuation:
        content_string = content_string.replace(word, " ")
    return(content_string)

def remove_stopwords_from_dict(word_dict, stopwords):
    for w in stopwords:
        word_dict.pop(w, word_dict)
    return word_dict

def lcut_to_dict(lcut):
    word_dict = dict(Counter(lcut))
#     word_dict.pop(' ')
    return(remove_stopwords_from_dict(word_dict, stopwords))

def sort_dict_by_values(d):
    return(sorted(d.items(), key=lambda x: x[1], reverse=True))

In [8]:
def news_containing_keyword(keyword, news_list):
    return list(filter(lambda news: keyword in news, news_list))  

def data_containing_keyword(keyword, data):
    return list(filter(lambda news: keyword in news['cutted_dict'].keys(), data))

def news_containing_keywords(keywords, news_list):
    news = news_list
    for keyword in keywords:
        news = news_containing_keyword(keyword, news)
        
    return news

# define the data_set and make it into a dict

In [9]:
def get_news_info(news_data,news_list) :    
    for i in range(len(data)) :
        tpye = data.iloc[i,0]
        cate = data.iloc[i,1]
        date = datetime.strftime(data.iloc[i,2],'%Y-%m-%d')
        title = data.iloc[i,3]
        content = data.iloc[i,4]
    
        info = {'type' : tpye
               ,'cate' : cate
               ,'date' : date
               ,'title': title
               ,'content': content}
        news_list.append(info)
    return (news_list)


In [10]:
news_list = []
each_news_info = get_news_info(data,news_list)
each_news_info[0]

{'type': 1,
 'cate': '全部財經新聞',
 'date': '2016-01-01',
 'title': '外資2015年台股買超462億元 四年來最少',
 'content': '外資2015年在台股買超462億元，是四年來最少金額，但在亞股中已是買超最多的市場，使得外資在台股持有市值比重達38.6%，展望2016年，外資能否重啟買超動能，法人認為，端視台股基本面與國際經濟情勢而定。2008年發生金融海嘯後，外資除遇重大金融危機，皆是年年大手筆買超台股，僅在2011年歐債危機首度爆發時賣超台股2,776億元；2012年買超1,442億元、2013年買超2,430億元、2014年買超3,546億元、2015年的買超還不到500億元，已屬相對較少。在亞股中，外資對東南亞股市泰國、印尼、菲律賓皆是賣超，僅在台股與印度股市是買超，相對青睞台股。根據彭博統計，外資全年在南韓股市賣超36億美元，在台股卻是買超34億美億，冷熱對照，依然「重台股、輕韓股」，在台股買超金額也高於印度買超31億美元。華南永昌投顧董事長儲祥生說，外資上半年先大買，自六月後開始調節，先買後賣才會在台股全年還是買超，顯示台股還是不乏體質強健的優質公司，加上台股本益比不高、資本市場制度相對健全，且台幣貶值幅度有限，有利外資流入。不過，儲祥生擔心，美股走了六年多的多頭走勢，恐有回檔壓力，由於外資布局台股多是與美股連動，美股繼強才有利外資繼續前進台股。此外，由於MSCI明晟指數已經逐步將陸股納入指數，先從14檔在美股掛的ADR（美國存託憑證）著手，未來A股也可望納入，儲祥生說，除了陸股比重將提升外，陸企納入成分股的多是互聯網概念股，已非台股強項，整體展望還是保守為宜。'}

In [11]:
def get_cutted_dict(list_of_news):
    cat = ' '.join(list_of_news)
    cat = remove_punctuation(cat)
    cutted = jieba.lcut(cat)
    return lcut_to_dict(cutted)

def first_n_words(cutted_dict, n, word_len=2, to=1000):
    sdbv = sort_dict_by_values(cutted_dict)
    return list(filter(lambda x: len(x[0])>=word_len and len(x[0])<=to, sdbv))[:n]

In [12]:
# add cutted dict to each news
for i in range(len(each_news_info)):
    current_content = each_news_info[i]['content']
    current_cutted = jieba.lcut(remove_punctuation(current_content))
    each_news_info[i]['cutted_dict'] = lcut_to_dict(current_cutted)

## classified the data_set into different tpye we want 

for example : we want all bank data

In [13]:
bank_dict = []
def filter_word (each_news_info,filer = '銀行'):
    for i in range(len(each_news_info)):
        if '銀行' in each_news_info[i]['content']:
            bank_dict.append(each_news_info[i])
    return bank_dict

In [14]:
bank_dict = filter_word(each_news_info,filer='銀行')

In [15]:
bank_dict[0]

{'type': 7,
 'cate': '全部財經新聞',
 'date': '2016-01-01',
 'title': '國銀大陸分行 再爆逾放',
 'content': '國銀大陸分行再爆逾放，新增台銀一家，金管會昨（31）日公布最新國銀營運概況，大陸分行接連兩月暴增逾放，使得出現逾放的銀行家數增加到三家，是台資銀登陸以來最多，顯見大陸授信風險升高。金管會銀行局副局長邱淑貞表示，台銀踩到的逾放，跟土銀是同一家從事工具機的台商公司，目前知道的有三家台資銀行都有貸款給這家台商，除了台銀、土銀外，另一家是富邦金大陸子行華一銀行。 對於國銀大陸分行逾放一家家爆，邱淑貞表示，銀行到國外開拓市場，是機會也是風險，銀行要注意風控，有逾放出來更要進一步加強風險控管。金管會昨天發布本國銀行經營概況，到2015年11月底止，國銀大陸分行逾放出現增加情況，除了2014年就有逾放的合庫外，及2015年10月新增的土銀外，又跑出台銀，逾放金額3,600萬元。國銀登陸營運以來，合庫大陸分行首度出現逾放，2015年4月國泰世華銀行大陸分行也踩到逾放，10月及11月接連新增土銀及台銀兩家，一年內首度連續有三家國銀大陸分行踩到逾放。其中國泰世華銀行已打掉，大陸分行目前並無逾放。銀行局表示，台銀跟土銀這筆放款對象，都是貸款給一家從事工具機的台商公司，因經營不佳，還不出錢來，變成逾放。根據金管會最新資料，到11月底，台銀大陸分行逾放金額3,600萬元，占放款比率為0.48%，這是台銀大陸分行第一次出現逾放。銀行業者表示，大陸經濟成長放緩，授信風險升高，不少銀行已減少對大陸地區放款，如今國銀大陸分行接連踩到逾放，也讓國銀授信更趨謹慎。對於國銀大陸分行逾放增加，金管會強調已接連採取多項措施，要求銀行加強控管大陸地區風險，也對大陸曝險控管加強金檢。此外，在國銀獲利方面，2015年前11個月獲利2,972億元，仍比2014年衰退2.5%，其中大陸分行減幅最大，衰退23%，國際金融業務分行（OBU）也衰退11%。',
 'cutted_dict': {'國銀': 8,
  '大陸': 19,
  '分行': 14,
  '再': 1,
  '爆': 2,
  '逾放': 18,
  '新增': 3,
  '台銀': 6,
  '一家': 3,
  '金管會': 5,
  '昨': 1,
  '日': 

# colculate the tf and df 

In [16]:
def merge_one_day_news_dict(one_day_dict, count='wt', divide = 1):
    all_words = set([word for each_dict in one_day_dict for word in each_dict])
    one_day_wf = {}
    for word in all_words:
        one_day_wf[word] = 0
        for news in one_day_dict:
            if count == 'wt':
                one_day_wf[word] += float(news.get(word, 0) * divide)
            if count == 'occur':
                one_day_wf[word] += float(bool(news.get(word, 0)) * divide)
    return one_day_wf

# sum the news a day and total terms a day

In [17]:
date_list = [news['date'] for news in each_news_info]
all_date = sorted(list(set(date_list)))
#aall_date = [date[5:] for date in all_date]
date_index = sorted([date_list.index(each_date) for each_date in all_date])
date_index.append(len(date_list)-1)
number_of_news = [date_index[i+1] - date_index[i]-1 for i in range(len(date_index)-1)]
number_of_terms = [sum([sum(each_news_info[ni]['cutted_dict'].values()) for ni in range(date_index[i], date_index[i+1])]) for i in range(len(date_index)-1)]

In [18]:
# 每日詞數
wn_by_day = {}
for i in range(len(date_index)-1):
    oneday_news = each_news_info[date_index[i]: date_index[i+1]]
    oneday_dict = [news['cutted_dict'] for news in oneday_news]
    current_word_dict = merge_one_day_news_dict(oneday_dict)
#     print(all_date[i])
    wn_by_day[all_date[i]] = current_word_dict

In [19]:
# 每日詞頻
tf_by_day = {}
for i in range(len(date_index)-1):
    oneday_news = each_news_info[date_index[i]: date_index[i+1]]
    oneday_dict = [news['cutted_dict'] for news in oneday_news]
    current_word_dict = merge_one_day_news_dict(oneday_dict, divide=number_of_terms[i])
#     print(all_date[i])
    tf_by_day[all_date[i]] = current_word_dict

In [20]:
# 每天出現的字
occur_by_day = {}
for i in range(len(date_index)-1):
    oneday_news = each_news_info[date_index[i]: date_index[i+1]]
    oneday_dict = [news['cutted_dict'] for news in oneday_news]
    current_word_dict = merge_one_day_news_dict(oneday_dict, count='occur', divide=number_of_news[i])
#     print(all_date[i])
    occur_by_day[all_date[i]] = current_word_dict

In [21]:
def first_n_wf_by_day(wf_by_day, n, word_len=2, to=1000):
    return {day: dict(first_n_words(wf_by_day[day], n, word_len=word_len, to=to)) for day in wf_by_day}

In [34]:
contents = []
for news in each_news_info :
    contents.append(news['content'])

# 可能的經濟事件

In [37]:
cutted_dict = get_cutted_dict(contents[:1000])
high_freq_pair = first_n_words(cutted_dict,100,4,10)
high_freq_pair

[('中華民國', 105),
 ('基本工資', 77),
 ('晴時多雲', 74),
 ('沙烏地阿', 73),
 ('電視辯論', 71),
 ('中國國民黨', 51),
 ('日本政府', 43),
 ('今天下午', 40),
 ('政見發表', 40),
 ('升旗典禮', 39),
 ('今天上午', 32),
 ('總統大選', 31),
 ('兩岸關係', 27),
 ('馬來西亞', 23),
 ('諾貝爾獎', 23),
 ('公共電視', 22),
 ('巴基斯坦', 22),
 ('虛擬實境', 20),
 ('高速公路', 17),
 ('達成協議', 17),
 ('再生能源', 17),
 ('最低工資', 17),
 ('中獎號碼', 16),
 ('恐怖主義', 15),
 ('恐怖分子', 15),
 ('外交關係', 15),
 ('股價指數', 14),
 ('維持現狀', 14),
 ('元旦假期', 14),
 ('產業政策', 14),
 ('有價證券', 14),
 ('中華電信', 13),
 ('去年同期', 13),
 ('臺灣海峽', 13),
 ('核電機組', 13),
 ('中國時報', 13),
 ('蘋果日報', 13),
 ('未成年人', 13),
 ('空軍基地', 13),
 ('立法委員', 13),
 ('中華會館', 13),
 ('美元匯率', 13),
 ('上證綜指', 13),
 ('一段時間', 12),
 ('特拉維夫', 12),
 ('中央銀行', 12),
 ('哥倫比亞', 12),
 ('新年快樂', 11),
 ('外交部長', 11),
 ('國際機場', 11),
 ('主管機關', 11),
 ('有限公司', 11),
 ('巴士海峽', 11),
 ('競爭對手', 11),
 ('中小企業', 10),
 ('航空公司', 10),
 ('解決方案', 10),
 ('國定假日', 10),
 ('經濟部長', 10),
 ('第二炮兵', 10),
 ('工作人員', 10),
 ('外匯市場', 10),
 ('個人電腦', 9),
 ('中央氣象局', 9),
 ('戶政事務', 9),
 ('汰舊換新', 9),
 ('主辦單位', 9

# 可能的名稱

In [52]:
cutted_dict = get_cutted_dict(contents[:1000])
high_freq_name = first_n_words(cutted_dict,100,2,3)
high_freq_name

[('多雲', 1400),
 ('台灣', 787),
 ('陣風', 750),
 ('總統', 734),
 ('短暫', 659),
 ('大陸', 515),
 ('去年', 459),
 ('局部', 457),
 ('美國', 408),
 ('市場', 407),
 ('候選人', 407),
 ('英文', 380),
 ('中國', 352),
 ('指出', 335),
 ('未來', 320),
 ('經濟', 318),
 ('政府', 305),
 ('民眾', 302),
 ('民進黨', 302),
 ('公尺', 297),
 ('成長', 288),
 ('浪高', 279),
 ('國民黨', 276),
 ('發展', 275),
 ('國際', 273),
 ('報導', 254),
 ('希望', 247),
 ('產業', 246),
 ('立倫', 244),
 ('包括', 237),
 ('公司', 232),
 ('小浪', 230),
 ('東北風', 228),
 ('時陰', 224),
 ('全球', 223),
 ('時間', 220),
 ('國家', 220),
 ('億元', 213),
 ('日本', 213),
 ('政策', 209),
 ('陰時', 201),
 ('影響', 193),
 ('持續', 192),
 ('企業', 192),
 ('超過', 188),
 ('兩岸', 188),
 ('指數', 179),
 ('推動', 173),
 ('浪轉', 169),
 ('活動', 166),
 ('增加', 166),
 ('開放', 161),
 ('元旦', 158),
 ('投資', 156),
 ('美元', 156),
 ('地區', 154),
 ('相關', 153),
 ('一個', 152),
 ('透過', 151),
 ('工作', 150),
 ('海面', 149),
 ('中浪', 149),
 ('支持', 148),
 ('股市', 145),
 ('勞工', 144),
 ('社會', 144),
 ('伊朗', 143),
 ('顯示', 141),
 ('表現', 141),
 ('台股', 139),
 ('提供', 138),
 

In [32]:
df = pd.DataFrame(wn_by_day)
df = df.fillna(0)

df_tf = pd.DataFrame(tf_by_day)
df_tf = df_tf.fillna(0)

df_occur = pd.DataFrame(occur_by_day)
df_occur = df_occur.fillna(0)

In [53]:
events = []
for name in high_freq_pair :
    events.append(name[0])
names = []
for name in high_freq_name :
    names.append(name[0])

In [54]:
# each event with tfdf of related people
event_people_dict = {}
for event in events:
    event_data = data_containing_keyword(event, each_news_info)
    # event_data[0]
    name_times_of_date = {}
    for date in all_date:
        oneday_data = list(filter(lambda data: data['date']==date, event_data))
        oneday_dict = [news['cutted_dict'] for news in oneday_data]
        current_word_dict = merge_one_day_news_dict(oneday_dict, 'wt', divide=len(oneday_data))
        possible_name = first_n_words(current_word_dict, 1000, 2, 3)
        name_times_of_date[date]= list(filter(lambda x: x[0] in names, possible_name))

    all_names = set([name for name_list in name_times_of_date.values() for (name, times) in name_list])
    event_people_dict[event] = {name: [dict(name_times_of_date[date]).get(name, 0) for date in all_date] for name in all_names}
    
# with open('../data/event_people_dict.json', 'w', encoding='utf-8') as j:
#     json.dump(event_people_dict, j)

In [56]:
pd.DataFrame(event_people_dict).T.head(10)

,一個,上午,下午,下跌,中國,中浪,人民,企業,伊朗,候選人,...,計畫,超過,透過,選舉,銀行,開放,陣雨,陣風,陰時,顯示
中華民國,"[117.0, 16.0, 0, 40.0, 4.0, 16.0, 35.0, 15.0, ...","[52.0, 0, 10.0, 24.0, 8.0, 0, 7.0, 0, 0, 6.0, ...","[0, 8.0, 0, 0, 0, 8.0, 7.0, 0, 0, 12.0, 0, 12....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[65.0, 8.0, 5.0, 32.0, 4.0, 40.0, 49.0, 15.0, ...",NaN,"[169.0, 32.0, 0, 0, 4.0, 0, 14.0, 5.0, 0, 6.0,...","[0, 0, 0, 16.0, 0, 0, 0, 0, 0, 0, 0, 12.0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[78.0, 64.0, 10.0, 48.0, 16.0, 24.0, 21.0, 70....",...,"[0, 8.0, 15.0, 0, 0, 0, 7.0, 0, 0, 0, 0, 60.0,...","[442.0, 0, 5.0, 0, 0, 16.0, 7.0, 0, 0, 6.0, 3....","[0, 8.0, 5.0, 0, 0, 8.0, 0, 0, 27.0, 0, 0, 12....","[0, 24.0, 5.0, 40.0, 8.0, 8.0, 7.0, 20.0, 0, 0...","[0, 0, 0, 16.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[364.0, 16.0, 0, 8.0, 0, 8.0, 0, 5.0, 18.0, 0,...",NaN,NaN,NaN,"[104.0, 0, 5.0, 0, 0, 32.0, 0, 5.0, 0, 0, 0, 0..."
基本工資,"[16.0, 60.0, 8.0, 25.0, 12.0, 1.0, 0, 6.0, 0, ...","[0, 0, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 18.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4.0, 0, 0, 0, 0, 0, 0, 4.0, 0, 0, 0, 0, 0, 0,...",NaN,"[20.0, 18.0, 0, 5.0, 12.0, 0, 0, 0, 0, 0, 0, 0...","[0, 12.0, 0, 10.0, 8.0, 0, 0, 0, 4.0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4.0, 120.0, 44.0, 75.0, 36.0, 2.0, 3.0, 16.0,...",...,"[0, 6.0, 0, 5.0, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[56.0, 6.0, 8.0, 0, 4.0, 0, 0, 0, 0, 0, 0, 0, ...","[4.0, 6.0, 4.0, 10.0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 60.0, 4.0, 40.0, 8.0, 0, 0, 4.0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[48.0, 6.0, 0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0, 0,...",NaN,NaN,NaN,"[12.0, 0, 12.0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0, 0..."
晴時多雲,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 14.0, 0, 0, 0, 0, 0, 0, 0, ...","[24.0, 4.0, 2.0, 3.0, 9.0, 0, 70.0, 12.0, 14.0...",NaN,NaN,"[160.0, 48.0, 16.0, 7.0, 36.0, 0, 259.0, 104.0...",NaN,NaN,NaN,NaN,...,NaN,"[0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 2.0, 0, 0, 0, 0, 0...",NaN,NaN,"[8.0, 8.0, 4.0, 5.0, 27.0, 0, 0, 12.0, 273.0, ...","[704.0, 304.0, 50.0, 30.0, 195.0, 0, 1659.0, 4...","[72.0, 92.0, 24.0, 1.0, 30.0, 8.0, 56.0, 12.0,...",NaN
沙烏地阿,"[24.0, 0, 0, 15.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 30.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 9.0, 135.0, 70.0, 8.0, 0, 2.0, 0, 0, 3....","[6.0, 0, 18.0, 105.0, 80.0, 0, 10.0, 2.0, 0, 0...",NaN,"[30.0, 0, 18.0, 0, 10.0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 10.0, 0, 0, 0, 0, 0, 0, 3.0, 0, 0...","[9.0, 0, 333.0, 960.0, 230.0, 60.0, 145.0, 7.0...","[6.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",...,"[0, 0, 0, 15.0, 0, 4.0, 0, 0, 0, 0, 0, 0, 0, 0...","[84.0, 0, 0, 120.0, 40.0, 0, 0, 0, 0, 0, 0, 6....","[6.0, 0, 9.0, 0, 10.0, 0, 0, 1.0, 0, 0, 0, 0, ...","[0, 0, 0, 15.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[72.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,"[18.0, 0, 9.0, 45.0, 30.0, 16.0, 5.0, 1.0, 0, ..."
電視辯論,"[0, 680.0, 5.0, 8.0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 3.0, 1.0, 0, 0, 0, 0, 0, 0, 0,...","[3.0, 560.0, 0, 0, 0, 3.0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3.0, 360.0, 30.0, 0, 0, 3.0, 0, 0, 0, 0, 0, 0...",NaN,"[6.0, 1440.0, 5.0, 8.0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 360.0, 0, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[36.0, 5120.0, 65.0, 48.0, 1.0, 33.0, 2.0, 8.0...",...,"[0, 160.0, 0, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 240.0, 5.0, 0, 0, 3.0, 0, 0, 0, 0, 0, 0, 0...","[3.0, 200.0, 10.0, 8.0, 0, 0, 0, 2.0, 0, 0, 0,...","[15.0, 1280.0, 20.0, 8.0, 0, 18.0, 2.0, 2.0, 0...","[0, 80.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 800.0, 15.0, 4.0, 1.0, 0, 0, 0, 0, 0, 0, 0...",N

In [58]:
def get_tfdf(word, df_tf, df_occur):
    tfdf = df_occur.loc[word] * df_tf.loc[word]
    return tfdf

def get_high_tfdf_date(word, df_tf, df_occur):
    tfdf = get_tfdf(word, df_tf, df_occur)
    m = statistics.mean(tfdf)
    s = statistics.stdev(tfdf)
    tfdf_bool = [x > m+s for x in tfdf]
    tfdf_date = {all_date[x[0]]: tfdf[x[0]] for x in list(filter(lambda e: e[1], enumerate(tfdf_bool)))}
    return tfdf_date


In [59]:
people_tfdf_dict = {name: list(get_tfdf(name, df_tf, df_occur)) for name in names}

In [61]:
pd.DataFrame(people_tfdf_dict, index=all_date).T.head(4)

,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,...,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30
多雲,2.591968e+10,1.001170e+10,371778000.0,0.0,5.314004e+11,2.614444e+10,1038768.0,9.886569e+10,1.226780e+10,5.803691e+10,...,4.182521e+07,0.000000e+00,0.000000e+00,2.587866e+07,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.000000e+00
台灣,5.670832e+10,3.775964e+10,300238900.0,1392.0,1.749998e+12,8.119571e+10,6816915.0,9.787281e+11,7.519031e+10,7.871152e+10,...,2.193314e+12,2.002588e+12,4.964117e+11,7.024503e+11,2.091251e+10,21528.0,15921472.0,1.405400e+13,660409600.0,1.784104e+13
陣風,4.334394e+09,2.244002e+09,274890400.0,0.0,1.780537e+11,9.310315e+09,0.0,6.115730e+10,6.699437e+09,1.002695e+10,...,4.182521e+07,1.689164e+08,1.819358e+07,2.587866e+07,0.000000e+00,0.0,0.0,1.091234e+09,1086200.0,9.272588e+08
總統,5.479396e+10,7.318898e+10,136881900.0,0.0,8.241020e+11,2.266586e+10,0.0,2.990617e+11,2.009831e+10,5.543468e+10,...,2.553011e+11,4.419275e+11,7.412062e+10,6.987237e+10,2.323612e+09,0.0,11372480.0,3.732019e+12,58654800.0,4.927221e+12
